In [ ]:
!git clone https://github.com/UniboSecurityResearch/PLC-LD-dataset


Cloning into 'PLC-LD-dataset'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (333/333), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 333 (delta 299), reused 279 (delta 256), pack-reused 0 (from 0)
Receiving objects: 100% (333/333), 181.95 KiB | 991.00 KiB/s, done.
Resolving deltas: 100% (299/299), done.


In [ ]:
!pip install -q transformers datasets accelerate torch xmltodict evaluate peft optuna wandb scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 2. Imports & Config
import os
import xmltodict
import numpy as np
from dataclasses import dataclass
from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed
)
import wandb

@dataclass
class Config:
    legit_dir: str = "/content/PLC-LD-dataset/legitimate"
    mal_dir: str   = "/content/PLC-LD-dataset/malicious"
    model_name: str = "microsoft/codebert-base"
    output_dir: str = "/content/model_output"
    max_length: int = 512
    train_batch_size: int = 16
    eval_batch_size: int  = 32
    epochs: int     = 100
    lr: float       = 2e-5
    weight_decay: float = 0.01
    seed: int       = 42
    eval_split: float = 0.2
    accum_steps: int = 2
    fp16: bool      = True
    logging_steps: int = 100
    save_steps: int    = 500
    project_name: str = "PLC-LD-Detection"

cfg = Config()
set_seed(cfg.seed)

# Initialize Weights & Biases
wandb.init(project=cfg.project_name, config=vars(cfg))

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kamaleshfine (kamaleshfine-xx) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# 3. Load and Prepare Data

def xml_to_text(path):
    with open(path, 'r', encoding='utf-8') as f:
        return str(xmltodict.parse(f.read()))

texts, labels = [], []
for fn in os.listdir(cfg.legit_dir):
    if fn.endswith('.xml') and fn.startswith('l'):
        legit_path = os.path.join(cfg.legit_dir, fn)
        mal_path   = os.path.join(cfg.mal_dir, 'm'+fn[1:])
        if os.path.exists(mal_path):
            texts.append(xml_to_text(legit_path)); labels.append(0)
            texts.append(xml_to_text(mal_path));   labels.append(1)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=cfg.eval_split, random_state=cfg.seed, stratify=labels
)

dataset = Dataset.from_dict({
    'text': train_texts + val_texts,
    'label': train_labels + val_labels
})

# 4. Tokenization

tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

def preprocess(examples):
    enc = tokenizer(
        examples['text'], padding='max_length', truncation=True,
        max_length=cfg.max_length
    )
    enc['labels'] = examples['label']
    return enc

dataset = dataset.map(preprocess, batched=True)

dataset = dataset.train_test_split(test_size=len(val_texts), seed=cfg.seed)
train_dataset = dataset['train']
eval_dataset  = dataset['test']

data_collator = DataCollatorWithPadding(tokenizer)

# 5. Metrics
metric_acc = evaluate.load('accuracy')
metric_f1  = evaluate.load('f1')
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return {
        **metric_acc.compute(predictions=preds, references=labels),
        **metric_f1.compute(predictions=preds, references=labels, average='weighted')
    }

# 6. TrainingArguments & Trainer
training_args = TrainingArguments(
    output_dir=cfg.output_dir,
    num_train_epochs=cfg.epochs,
    per_device_train_batch_size=cfg.train_batch_size,
    per_device_eval_batch_size=cfg.eval_batch_size,
    gradient_accumulation_steps=cfg.accum_steps,
    learning_rate=cfg.lr,
    weight_decay=cfg.weight_decay,
    fp16=cfg.fp16,
    logging_steps=cfg.logging_steps,
    save_steps=cfg.save_steps,
    seed=cfg.seed,
    report_to="wandb"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(cfg.model_name, num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2724792132.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 7. Train & Evaluate
trainer.train()
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
100,0.445100
200,0.319000


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Evaluation Results: {'eval_loss': 0.3746299743652344, 'eval_accuracy': 0.8333333333333334, 'eval_f1': 0.8333333333333334, 'eval_runtime': 4.5221, 'eval_samples_per_second': 2.654, 'eval_steps_per_second': 0.221, 'epoch': 100.0}


In [ ]:
model.save_pretrained(cfg.output_dir)
tokenizer.save_pretrained(cfg.output_dir)
print(f"Model and tokenizer saved to {cfg.output_dir}")

Model and tokenizer saved to /content/model_output


In [ ]:
import pandas as pd
from IPython.display import display

eval_df = pd.DataFrame([eval_results])
display(eval_df)


,eval_loss,eval_accuracy,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.37463,0.833333,0.833333,4.5221,2.654,0.221,100.0


In [ ]:
!zip -r model_output.zip /content/model_output

  adding: content/model_output/ (stored 0%)
  adding: content/model_output/model.safetensors (deflated 7%)
  adding: content/model_output/checkpoint-200/ (stored 0%)
  adding: content/model_output/checkpoint-200/model.safetensors (deflated 7%)
  adding: content/model_output/checkpoint-200/scaler.pt (deflated 60%)
  adding: content/model_output/checkpoint-200/merges.txt (deflated 53%)
  adding: content/model_output/checkpoint-200/special_tokens_map.json (deflated 84%)
  adding: content/model_output/checkpoint-200/trainer_state.json (deflated 58%)
  adding: content/model_output/checkpoint-200/optimizer.pt (deflated 36%)
  adding: content/model_output/checkpoint-200/tokenizer.json (deflated 82%)
  adding: content/model_output/checkpoint-200/config.json (deflated 51%)
  adding: content/model_output/checkpoint-200/scheduler.pt (deflated 56%)
  adding: content/model_output/checkpoint-200/rng_state.pth (deflated 25%)
  adding: content/model_output/checkpoint-200/tokenizer_config.json (deflate